# HAR Model
For each risk classficiation, we will train a model to fit to predict the RV model

## Import the libraries and data
To obtain the data, please go to notebooks/data_preprocessing, and then run data_import.ipynb and then run data_preprocessing.ipynb. This will give you data/processed_data.csv

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
from arch import arch_model

hourly_data = pd.read_csv('../..//data/hourly_data.csv')
three_hourly_data = pd.read_csv('../../data/three_hourly_data.csv')
daily_data = pd.read_csv('../../data/daily_data.csv')

## Train test split
Now we will use a different train-test split from the group project
Group project: 80/20 split
Individual: Use 1 year of training data, then use rolling window 

In [28]:

# Train-test split
# Sort the data by date
hourly_data['Date'] = pd.to_datetime(hourly_data['Date'])
three_hourly_data['Date'] = pd.to_datetime(three_hourly_data['Date'])
daily_data['Date'] = pd.to_datetime(daily_data['Date'])

hourly_data = hourly_data.sort_values('Date')

# Determine when the first year ends, and use it as train data
# The rest of the data is used as test data
min_date = hourly_data['Date'].min()
max_date = hourly_data['Date'].max()

# Calculate the total time span of the data
total_time_span = max_date - min_date

# Define the first year of data
first_year_end = min_date + pd.DateOffset(years=1)

# Filter data for the first year
first_year_data = hourly_data[hourly_data['Date'] <= first_year_end]

# Calculate the percentage of data in the first year
percentage_first_year = (len(first_year_data) / len(hourly_data))

train_split = percentage_first_year

# Hourly data train-test split
X_train_hourly = hourly_data[hourly_data['Date'] <= first_year_end]
X_test_hourly = hourly_data[hourly_data['Date'] > first_year_end]

# Three hourly data train-test split
X_train_three_hourly = three_hourly_data[three_hourly_data['Date'] <= first_year_end]
X_test_three_hourly = three_hourly_data[three_hourly_data['Date'] > first_year_end]

# Daily data train-test split
X_train_daily = daily_data[daily_data['Date'] <= first_year_end]
X_test_daily = daily_data[daily_data['Date'] > first_year_end]

In [29]:
train_date_split_hourly = {
    'low': X_train_hourly[X_train_hourly['Risk'] == 'Low Risk'],
    'medium': X_train_hourly[X_train_hourly['Risk'] == 'Medium Risk'],
    'high': X_train_hourly[X_train_hourly['Risk'] == 'High Risk']
}

test_date_split_hourly = {
    'low': X_test_hourly[X_test_hourly['Risk'] == 'Low Risk'],
    'medium': X_test_hourly[X_test_hourly['Risk'] == 'Medium Risk'],
    'high': X_test_hourly[X_test_hourly['Risk'] == 'High Risk']
}

train_date_split_three_hourly = {
    'low': X_train_three_hourly[X_train_three_hourly['Risk'] == 'Low Risk'],
    'medium': X_train_three_hourly[X_train_three_hourly['Risk'] == 'Medium Risk'],
    'high': X_train_three_hourly[X_train_three_hourly['Risk'] == 'High Risk']
}

test_date_split_three_hourly = {
    'low': X_test_three_hourly[X_test_three_hourly['Risk'] == 'Low Risk'],
    'medium': X_test_three_hourly[X_test_three_hourly['Risk'] == 'Medium Risk'],
    'high': X_test_three_hourly[X_test_three_hourly['Risk'] == 'High Risk']
}

train_date_split_daily = {
    'low': X_train_daily[X_train_daily['Risk'] == 'Low Risk'],
    'medium': X_train_daily[X_train_daily['Risk'] == 'Medium Risk'],
    'high': X_train_daily[X_train_daily['Risk'] == 'High Risk']
}

test_date_split_daily = {
    'low': X_test_daily[X_test_daily['Risk'] == 'Low Risk'],
    'medium': X_test_daily[X_test_daily['Risk'] == 'Medium Risk'],
    'high': X_test_daily[X_test_daily['Risk'] == 'High Risk']
}


# Train the model based on their classifications

### Training data

This wil give us 3 models to work with: model_low, model_medium, and model_high. We will use these subsequent models on the test data to evaluate the models

In [ ]:
def train_models_by_frequency_and_risk(train_data, frequencies):
    models = {}
    model_summary = pd.DataFrame(columns=['Frequency', 'Risk Group', 'Omega', 'Alpha[1]', 'Beta[1]', 'Mean'])
    
    for freq in frequencies:
        models[freq] = {}
        if freq == 'hourly':
            target_col = 'ln_hourly_rv'
        elif freq == '3hourly':
            target_col = 'ln_3_hourly_rv'
        elif freq == 'daily':
            target_col = 'ln_daily_rv'
        else:
            raise ValueError(f"Unsupported frequency: {freq}")
            
        for risk_group in ['low', 'medium', 'high']:
            df_train = train_data[risk_group].copy()
            if target_col not in df_train.columns:
                print(f"Missing {target_col} for {freq}-{risk_group}, skipping.")
                continue
            
            series = df_train[target_col].dropna()
            if len(series) < 2:
                print(f"Insufficient data for {freq}-{risk_group}, skipping.")
                continue
            
            # Fit GARCH(1,1) with constant mean on the ln-transformed series
            am = arch_model(series, mean='Constant', vol='GARCH', p=1, q=1, dist='normal', rescale=False)
            res = am.fit(disp='off')
            
            models[freq][risk_group] = {
                'model': res,
                'params': res.params,  # Contains omega, alpha[1], beta[1], mu
                'last_var': res.conditional_volatility.iloc[-1]**2  
            }
            
            pars = res.params
            row = {
                'Frequency': freq,
                'Risk Group': risk_group,
                'Omega': pars.get('omega', np.nan),
                'Alpha[1]': pars.get('alpha[1]', np.nan),
                'Beta[1]': pars.get('beta[1]', np.nan),
                'Mean': pars.get('mu', np.nan)
            }
            model_summary = pd.concat([model_summary, pd.DataFrame([row])], ignore_index=True)
    
    return models, model_summary

model_daily, summary_daily = train_models_by_frequency_and_risk(train_date_split_daily, ['daily'])
model_hourly, summary_hourly = train_models_by_frequency_and_risk(train_date_split_hourly, ['hourly'])
model_three_hourly, summary_three_hourly = train_models_by_frequency_and_risk(train_date_split_three_hourly, ['3hourly'])

# Merge the summaries
summary = pd.concat([summary_daily, summary_hourly, summary_three_hourly], ignore_index=True)

C:\Users\young\AppData\Local\Temp\ipykernel_28088\85775861.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model_summary = pd.concat([model_summary, pd.DataFrame([row])], ignore_index=True)
C:\Users\young\AppData\Local\Temp\ipykernel_28088\85775861.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model_summary = pd.concat([model_summary, pd.DataFrame([row])], ignore_index=True)
C:\Users\young\AppData\Local\Temp\ipykernel_28088\85775861.py:51: FutureWarning: The behavior of DataFrame con

### Model summary

In [31]:
print(summary)

  Frequency Risk Group     Omega  Alpha[1]   Beta[1]       Mean
0     daily        low  0.037582  0.057627  0.914335  -7.997750
1     daily     medium  0.246348  0.071943  0.732727  -7.484376
2     daily       high  0.200376  0.221807  0.650649  -6.879797
3    hourly        low  4.892120  0.163660  0.101320 -12.915812
4    hourly     medium  0.298307  0.013675  0.940431 -12.266809
5    hourly       high  0.023704  0.003597  0.993066 -11.660366
6   3hourly        low  0.458925  0.124402  0.700717 -10.775833
7   3hourly     medium  0.699855  0.127292  0.575893 -10.184531
8   3hourly       high  0.078554  0.060181  0.911660  -9.567104


### Testing data

#### Implementing rolling window
Rolling window is used for a one step ahead forecast. So we constantly update the lagged data with an update lagged data

In [32]:
def rolling_window_predictions(X_test, y_test, model, window_size=24, step_ahead=1):
    predictions = []
    actuals = []
    dates = []
    
    max_index = len(X_test) - step_ahead  # Ensure we have data for the forecast horizon
    for i in range(window_size, max_index + 1):
        # Use the target series within the current rolling window
        window_data = y_test.iloc[i - window_size : i]
        
        # Re-fit the GARCH(1,1) model on the current window.
        # We use the same specification as in training.
        am = arch_model(window_data, mean='Constant', vol='GARCH', p=1, q=1, dist='normal', rescale=False)
        res = am.fit(disp='off')
        
        # Forecast one step ahead using the re-fitted model.
        forecast = res.forecast(horizon=step_ahead, reindex=False)
        forecast_value = forecast.mean.iloc[-1, 0]
        predictions.append(forecast_value)
        
        # Get the actual value at the forecast horizon.
        actual_index = i + step_ahead - 1
        actual_value = y_test.iloc[actual_index]
        actuals.append(actual_value)
        
        # Capture the corresponding date from the test data.
        forecast_date = X_test['Date'].iloc[actual_index]
        dates.append(forecast_date)
    
    return np.array(predictions), np.array(actuals), np.array(dates)


#### Implementing an evaluatation function
This function evaluates the findings and puts it in a df for each ticker

In [ ]:
def evaluate_models_on_test_data(test_data_split, models, frequencies, window_size=24, step_ahead=1):
    evaluation_summary = pd.DataFrame(columns=['Frequency', 'Risk Group', 'Ticker', 'MSE', 'R²'])
    detailed_results = pd.DataFrame(columns=['Date', 'Ticker', 'Risk Group', 'Frequency', 'Predicted', 'Actual'])
    
    for freq in frequencies:
        # Define target and forecast horizon based on frequency.
        if freq == 'hourly':
            target = 'ln_hourly_rv'
        elif freq == '3hourly':
            target = 'ln_3_hourly_rv'
        elif freq == 'daily':
            target = 'ln_daily_rv'
        else:
            raise ValueError(f"Unsupported frequency: {freq}")
        
        for risk_group in ['low', 'medium', 'high']:
            if risk_group not in models[freq]:
                continue
            model_info = models[freq][risk_group]
            group_data = test_data_split[risk_group].copy()
            
            unique_tickers = group_data['Ticker'].unique()
            for ticker in unique_tickers:
                ticker_data = group_data[group_data['Ticker'] == ticker].copy()
                if target not in ticker_data.columns:
                    print(f"Skipping {ticker}: missing target {target} for {freq} - {risk_group}")
                    continue
                
                X_test = ticker_data[['Date', target]].dropna()
                y_test = X_test[target]
                if len(X_test) < window_size + step_ahead:
                    print(f"Skipping {ticker}: insufficient data for {freq}-{risk_group} ({len(X_test)} rows)")
                    continue
                
                # Generate forecasts using the GARCH forecast-based rolling window prediction function.
                predictions, actuals, dates = rolling_window_predictions(
                    X_test, y_test, model_info, window_size=window_size, step_ahead=step_ahead
                )
                
                valid_idx = ~np.isnan(predictions) & ~np.isnan(actuals)
                predictions = predictions[valid_idx]
                actuals = actuals[valid_idx]
                dates = dates[valid_idx]
                if len(predictions) == 0:
                    continue
                
                mse = mean_squared_error(actuals, predictions)
                r2 = r2_score(actuals, predictions)
                
                eval_row = pd.DataFrame({
                    'Frequency': [freq],
                    'Risk Group': [risk_group.capitalize()],
                    'Ticker': [ticker],
                    'MSE': [mse],
                    'R²': [r2]
                })
                evaluation_summary = pd.concat([evaluation_summary, eval_row], ignore_index=True)
                
                ticker_results = pd.DataFrame({
                    'Date': dates,
                    'Ticker': ticker,
                    'Risk Group': risk_group.capitalize(),
                    'Frequency': freq,
                    'Predicted': predictions,
                    'Actual': actuals
                })
                detailed_results = pd.concat([detailed_results, ticker_results], ignore_index=True)
    
    return evaluation_summary, detailed_results


#### Evaluate the test data

In [ ]:
evaluation_summary_hourly, detailed_results_hourly = evaluate_models_on_test_data(
    test_date_split_hourly, model_hourly, ['hourly'], window_size=24
)
evaluation_summary_three_hourly, detailed_results_three_hourly = evaluate_models_on_test_data(
    test_date_split_three_hourly, model_three_hourly, ['3hourly'], window_size=24
)
evaluation_summary_daily, detailed_results_daily = evaluate_models_on_test_data(
    test_date_split_daily, model_daily, ['daily'], window_size=24
)

# Combine the evaluation summaries across frequencies, if desired.
combined_summary = pd.concat([evaluation_summary_hourly, evaluation_summary_three_hourly, evaluation_summary_daily], ignore_index=True)
combined_details = pd.concat([detailed_results_hourly, detailed_results_three_hourly, detailed_results_daily], ignore_index=True)

# Save the detailed results to CSV
combined_details.to_csv('../../results/garch.csv', index=False)

# Print the evaluation summaries
print("Combined Evaluation Summary:")
print(combined_summary)


C:\Users\young\AppData\Local\Temp\ipykernel_28088\22485321.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  evaluation_summary = pd.concat([evaluation_summary, eval_row], ignore_index=True)
C:\Users\young\AppData\Local\Temp\ipykernel_28088\22485321.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  detailed_results = pd.concat([detailed_results, ticker_results], ignore_index=True)
C:\Users\young\AppData\Local\Temp\ipykernel_28088\22485321.py:62: FutureWarning: The behavior of DataFrame con

Combined Evaluation Summary:
                           Date   Ticker Risk Group Frequency  Predicted  \
0     2024-04-02 01:00:00+00:00  BTC-USD        Low    hourly -12.434889   
1     2024-04-02 02:00:00+00:00  BTC-USD        Low    hourly -12.596374   
2     2024-04-02 03:00:00+00:00  BTC-USD        Low    hourly -12.247775   
3     2024-04-02 04:00:00+00:00  BTC-USD        Low    hourly -12.224047   
4     2024-04-02 05:00:00+00:00  BTC-USD        Low    hourly -12.169358   
...                         ...      ...        ...       ...        ...   
56210 2025-03-05 00:00:00+00:00  SOL-USD       High     daily  -6.084079   
56211 2025-03-06 00:00:00+00:00  SOL-USD       High     daily  -6.074680   
56212 2025-03-07 00:00:00+00:00  SOL-USD       High     daily  -6.069487   
56213 2025-03-08 00:00:00+00:00  SOL-USD       High     daily  -5.977923   
56214 2025-03-09 00:00:00+00:00  SOL-USD       High     daily  -6.006356   

          Actual  
0     -14.731732  
1      -6.711456  
2